In [ ]:
import csv
import random
import pandas
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
import cv2
import os

In [ ]:
memedata = []
with open("../data/train.jsonl", "r") as f:
    for line in f:
        memedata.append(json.loads(line))

In [ ]:
meme_data = []
for elem in memedata:
    elem['label']=1
    meme_data.append(elem)

meme_paths = []
for elem in meme_data:
    meme_paths.append({'label': elem['label'],'path': elem['img'],}) 


# print(meme_paths)

In [ ]:
normal_img_data = []
import pandas as pd
df = pd.read_csv('../train.csv')
normal_img_data = df.to_dict(orient='records')


In [ ]:
for elem in normal_img_data:
    elem['path'] = elem['Image']
    elem['label'] = 0

normal_img_data2=[]
for elem in normal_img_data:
    normal_img_data2.append({'label' : 0, 'path' : elem['path']})


In [ ]:
img_data_list = meme_data + normal_img_data2
random.shuffle(img_data_list)

In [ ]:
print(len(img_data_list))

I have used this model from the internet

In [ ]:
import torch
final =[]
X = []
Y=[]
img_size = (224,224)
for elem in img_data_list:
    img_arr = cv2.imread(elem['path'])
    img_arr_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    resized_arr = cv2.resize(img_arr_rgb, img_size)
    transposed_tensor = np.transpose(resized_arr, (2, 0, 1))
    t = torch.tensor(transposed_tensor)
    X.append(t)

    Y.append(elem['label'])

X_final = np.array(X)
X_tensor = torch.tensor(X_final)

In [ ]:
import pickle
with open('../tensor.pkl', 'rb') as f:
    loaded_tensor = pickle.load(f)
with open('../label.pkl', 'rb') as f:
    Y = pickle.load(f)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # input channels=3, output channels=32, kernel_size=3x3
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer with kernel_size=2x2, stride=2
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)  # input channels=32, output channels=32, kernel_size=3x3
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # input channels=32, output channels=64, kernel_size=3x3
        self.dropout = nn.Dropout2d(p=0.4)  # Dropout layer with dropout probability 0.4
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # Fully connected layer with input features 64*28*28 and output features 128
        self.fc2 = nn.Linear(128, 2)  # Fully connected layer with input features 128 and output features 2 (for binary classification)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply first convolutional layer followed by ReLU activation
        x = self.pool(x)  # Apply max pooling
        x = F.relu(self.conv2(x))  # Apply second convolutional layer followed by ReLU activation
        x = self.pool(x)  # Apply max pooling
        x = F.relu(self.conv3(x))  # Apply third convolutional layer followed by ReLU activation
        x = self.pool(x)  # Apply max pooling
        x = self.dropout(x)  # Apply dropout
        x = x.view(-1, 64 * 28 * 28)  # Reshape the tensor
        x = F.relu(self.fc1(x))  # Apply first fully connected layer followed by ReLU activation
        x = self.fc2(x)  # Apply second fully connected layer
        return F.softmax(x, dim=1)  # Apply softmax activation for classification

# Instantiate the model
model = Net()

# Print model summary
print(model)

import torch.optim as optim

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.000001)

# Define the loss function
criterion = nn.CrossEntropyLoss()



In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F

image_tensors =loaded_tensor
print(image_tensors)
labels = torch.tensor(Y)
batch_size = 32
epochs = 5
for epoch in range(epochs):
    indices = torch.randperm(len(image_tensors))
    print(indices)
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i in range(0, len(image_tensors), batch_size):
        batch_indices = indices[i:i+batch_size]
        print(i)
        batch_images = torch.stack([image_tensors[idx] for idx in batch_indices])
        batch_labels = torch.tensor([labels[idx] for idx in batch_indices])

        optimizer.zero_grad()

        outputs = model(batch_images)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += batch_labels.size(0)
        correct_train += (predicted == batch_labels).sum().item()

    # Calculate average training loss and accuracy
    train_loss = running_loss / (len(image_tensors) / batch_size)
    train_accuracy = correct_train / total_train

    # Print progress
    print(f'Epoch [{epoch+1}/{epochs}], '
          f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

# Training finished
print('Finished Training')



In [ ]:
import torch
from PIL import Image
from torchvision import transforms

model_path = 'model.pkl'
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

def preprocess_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to the size expected by the model
        transforms.ToTensor(),           # Convert the image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])
    processed_image = transform(image)
    processed_image = processed_image.unsqueeze(0)
    return processed_image

def predict_image(model, processed_image):
    model.eval()
    with torch.no_grad():
        outputs = model(processed_image)
    return outputs

image_path = '../data/img/01235.png'

processed_image = preprocess_image(image_path)

outputs = predict_image(loaded_model, processed_image)

print(outputs)


In [ ]:
img = "../data/img/01245.png"

In [ ]:
import torch
img_size = (224,224)
img_arr = cv2.imread(img)
img_arr_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
resized_arr = cv2.resize(img_arr_rgb, img_size)
transposed_tensor = np.transpose(resized_arr, (2, 0, 1))
t = torch.tensor(transposed_tensor).float()

In [ ]:
import pickle
model_path = "model.pkl"
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

In [ ]:
loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(t)
print(outputs[0])

In [ ]:
import glob

image_file1=glob.glob('../data/img/*.png')
image_files2 = glob.glob('../images/*.jpg')



In [ ]:
def process_images(image_file):
    for i in image_file:
        actual_negative=0
        not_positive=0
        img_size = (224,224)
        img_arr = cv2.imread(i)
        img_arr_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
        resized_arr = cv2.resize(img_arr_rgb, img_size)
        transposed_tensor = np.transpose(resized_arr, (2, 0, 1))
        t = torch.tensor(transposed_tensor).float()
        with torch.no_grad():
            outputs = loaded_model(t)
            outputs = outputs.tolist()
        if outputs[0][0]>outputs[0][1]:
            actual_negative+=1
        else:
            not_positive +=1
    return actual_negative,not_positive

In [ ]:
actual_negative,not_positive=process_images(image_file1)



In [ ]:
actual_negative2,not_positive2=process_images(image_files2)

In [ ]:
accuracy = 1 - (actual_negative + not_positive )/len(image_file1)
print(accuracy)
accuracy = 1 - (actual_negative2 + not_positive2 )/len(image_file1)
print(accuracy)